In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns

from GoogleNews import GoogleNews
from newspaper import Article, Config
import nltk
import openpyxl
from gensim import corpora, models, matutils
from collections import defaultdict
from time import sleep
from datetime import datetime, timedelta
import random

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

from textblob import TextBlob, Word
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer

nltk_stops = stopwords.words()
nltk.download('punkt')

%matplotlib inline

# config will allow us to access the specified url 
# for which we are #not authorized. Sometimes we may 
# get 403 client error while parsing #the link to 
# download the article.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
list = []

def web_search_function(search_term):
    # list = []

    # Set up a list of user agents to iterate through
    user_agent_list = [
        ('Mozilla/5.0'),  # firefox
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15'),  # safari
        ('Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'),  # firefox
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0'),  # firefox
        ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.110 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'),  # firefox
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.91 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'), # chrome
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'), # firefox
        ]

    date_range = int(input("How many days do you want to go back?"))

    today = datetime.today()
    end_date = today
    start_date = today - timedelta(days=date_range)

    end = f"{end_date:%m/%d/%Y}"
    start = f"{start_date:%m/%d/%Y}"

    # #Pick a random user agent
    # user_agent = random.choice(user_agent_list)
    # config = Config()
    # config.browser_user_agent = user_agent
    try:
        #Pick a random user agent
        user_agent = random.choice(user_agent_list)
        config = Config()
        config.browser_user_agent = user_agent

        # Set up search term(s) and date range
        # Create variables for data storage results
        googlenews = GoogleNews(start=start, end=end, encode="utf-8", period=start_date, lang="en")
        
        # googlenews = GoogleNews(start=start, end=end)
        # googlenews.set_lang('en')
        # googlenews.set_encode('utf-8')
        # googlenews.set_period(str(date_range))
        # googlenews.set_time_range(start_date, end_date)

        googlenews.search(search_term)

        result = googlenews.result()
        df = pd.DataFrame(result)
        print(len(df))
        print(df.head())
        sleep(5)

        # for i in range(len(df)):
        #     googlenews.getpage(i)
        #     result = googlenews.result()
        #     df = pd.DataFrame(result)
        #     sleep(2)

        # if googlenews.response == 429:
        #     #Pick a random user agent
        #     user_agent = random.choice(user_agent_list)
        #     config = Config()
        #     config.browser_user_agent = user_agent

        #     # Set up search term(s) and date range
        #     # Create variables for data storage results
        #     googlenews = GoogleNews(start='05/01/2021', end='09/11/2021')
        #     googlenews = GoogleNews(start=start, end=end)
        #     googlenews.search(search_term)
        #     result = googlenews.result()
        #     df.append(result)
        #     print(len(df))
        #     print(df.head())

        for ind in df.index:
            dict = {}

            # Use the defined User Agent string to bypass Error: 403 codes
            article = Article(df['link'][ind], config=config)

            # Download the article return
            article.download()

            # Parse the article return
            article.parse()
            article.nlp()

            # Create new columns in the DataFrame to store data
            dict['Date'] = df['date'][ind]
            dict['Media'] = df['media'][ind]
            dict['Title'] = article.title
            dict['Article'] = article.text
            dict['Summary'] = article.summary
            # Append to list
            list.append(dict)
            sleep(5)

        # Create a new DataFrame for the appended list
        news_df=pd.DataFrame(list)
        # Output to excel
        news_df.to_excel("articles.xlsx")

    except:
        #Pick a random user agent
        user_agent = random.choice(user_agent_list)
        config = Config()
        config.browser_user_agent(user_agent)

        # Set up search term(s) and date range
        # Create variables for data storage results
        googlenews = GoogleNews(start='05/01/2021', end=end_date)
        # googlenews = GoogleNews(start=start_date, end=end_date)
        googlenews.search(search_term)
        result = googlenews.result()
        print(df.head())
        print(len(df))

        for i in range(len(df)):
            googlenews.getpage(i)
            result = googlenews.result()
            df = pd.DataFrame(result)
            sleep(5)

        for ind in df.index:
            dict = {}

            # Use the defined User Agent string to bypass Error: 403 codes
            article = Article(df['link'][ind], config=config)

            # Download the article return
            article.download()

            # Parse the article return
            article.parse()
            article.nlp()

            # Create new columns in the DataFrame to store data
            dict['Date']=df['date'][ind]
            dict['Media']=df['media'][ind]
            dict['Title']=article.title
            dict['Article']=article.text
            dict['Summary']=article.summary
            # Append to list
            list.append(dict)
            sleep(5)

        # Create a new DataFrame for the appended list
        news_df=pd.DataFrame(list)
        # Output to excel
        news_df.to_excel("articles.xlsx")


    

    # for ind in df.index:
    #     dict = {}

    #     # Use the defined User Agent string to bypass Error: 403 codes
    #     article = Article(df['link'][ind], config=config)

    #     # Download the article return
    #     article.download()

    #     # Parse the article return
    #     article.parse()
    #     article.nlp()

    #     # Create new columns in the DataFrame to store data
    #     dict['Date']=df['date'][ind]
    #     dict['Media']=df['media'][ind]
    #     dict['Title']=article.title
    #     dict['Article']=article.text
    #     dict['Summary']=article.summary
    #     # Append to list
    #     list.append(dict)
    #     sleep(5)
    # # Create a new DataFrame for the appended list
    # news_df=pd.DataFrame(list)
    # # Output to excel
    # news_df.to_excel("articles.xlsx")


search_term = str(input("What term(s) do you want to search for?"))
web_search_function(search_term)

10
                                               title  \
0  Afghanistan bar association head pleads for in...   
1  The Taliban destroyed Afghanistan's ancient Bu...   
2  Journalist Ahmad Baseer Ahmadi severely beaten...   
3  More than 50 evacuees from Afghanistan in New ...   
4  100 Days of Taliban: A chronology of events af...   

                              media         date datetime  \
0                        Jurist.org  14 mins ago      NaT   
1                          NBC News  17 mins ago      NaT   
2  Committee to Protect Journalists  27 mins ago      NaT   
3                              WHDH  47 mins ago      NaT   
4                    Republic World  47 mins ago      NaT   

                                                desc  \
0  The President of the Afghanistan Independent B...   
1  The Taliban destroyed Afghanistan's Bamiyan Bu...   
2  Washington, D.C., November 24, 2021 — The Tali...   
3  More than 50 evacuees from Afghanistan have ar...   
4  The Taliba

## Code Notes
### Working Code

In [3]:
# # Set up a list of user agents to iterate through
# user_agent_list = [
#     ('Mozilla/5.0'),  # firefox
#     ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15'),  # safari
#     ('Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'),  # firefox
#     ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'),  # chrome
#     ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0'),  # firefox
#     ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'),  # chrome
#     ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.110 Safari/537.36'),  # chrome
#     ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'),  # chrome
#     ('Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'),  # firefox
#     ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.91 Safari/537.36'),  # chrome
#     ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36'),  # chrome
#     ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36'),  # chrome
#     ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'), # chrome
#     ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'), # firefox
#     ]

# today = datetime.today()
# # today = f"{today:%m/%d/%Y}"
# print(today)
# end = today
# start = today - timedelta(days=180)
# end_date = f"{end:%m/%d/%Y}"
# start_date = f"{start:%m/%d/%Y}"
# print(start_date)
# print(end_date)

# #Pick a random user agent
# user_agent = random.choice(user_agent_list)
# config = Config()
# config.browser_user_agent = user_agent

In [4]:
# print(f"{today:%B %d, %Y}")
# print(f"{today:%m/%d/%Y}")

In [5]:
# # Set up search terms and date range
# # Create variables for data storage results
# googlenews=GoogleNews(start=start_date,end=end_date,)
# # googlenews.user_agent('Mozilla/5.0')
# googlenews.search('Afghanistan'); sleep(1)
# result=googlenews.result(); sleep(1)
# df=pd.DataFrame(result)
# print(df.head())
# print(len(result))

# # if googlenews.user_agent(user_agent)
# #     print("Test")

In [6]:
# # Iterate through Google News, get the page
# # results, and store them in a DataFrame
# for i in range(2,20):
#     googlenews.getpage(i) #; sleep(2)
#     result = googlenews.result()
#     df=pd.DataFrame(result)
#     sleep(1)

# print(df.head())